In [1]:
from fastai.vision.all import *
from fastai.callback.all import EarlyStoppingCallback

In [2]:
import warnings

warnings.filterwarnings(
    "ignore",
    message="User provided device_type of 'cuda', but CUDA is not available.*"
)

In [3]:
from datasets import load_dataset
ds = load_dataset("uoft-cs/cifar10")
ds

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 10000
    })
})

In [4]:
from fastai.torch_core import defaults
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
defaults.device = device
device

device(type='cpu')

In [5]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [6]:
hf_train = ds['train'].with_format("torch")
hf_valid = ds['test'].with_format("torch")

In [7]:
class Ensure3Channels(Transform):
    def encodes(self, x:TensorImage):
        if x.ndim==3 and x.shape[0]==1:
            return x.repeat(3,1,1).contiguous()
        return TensorImage(x)

In [8]:
from torch.utils.data import Dataset

class HFtoFAI(Dataset):
    def __init__(self, hf_ds): self.ds = hf_ds
    def __len__(self): return len(self.ds)
    def __getitem__(self, i):
        ex = self.ds[i]
        x = TensorImage(ex['img'])
        y = int(ex['label'])
        return x, y


In [9]:
from torch.utils.data import Subset
from fastai.vision.all import *
import torch
from fastai.torch_core import defaults


item_tfms  = [Ensure3Channels(), Resize(32)]
batch_tfms = [IntToFloatTensor(),
              *aug_transforms(size=32, max_rotate=15, max_zoom=1.1, p_affine=0.75, p_lighting=0.8),
             Normalize.from_stats(*imagenet_stats)]

valid_batch_tfms = [IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)]

fold_metrics = []
learners = []

for fold, (train_idx, valid_idx) in enumerate(skf.split(hf_train['img'], hf_train['label'])):
    print(f'==== Fold {fold+1} ====')
    train_ds = Subset(hf_train, train_idx)
    valid_ds = Subset(hf_train, valid_idx)

    train_dl = TfmdDL(
        HFtoFAI(train_ds), bs=128, shuffle=True,
        after_item=item_tfms, after_batch=batch_tfms,
        num_workers=1, device='cpu',
    )

    valid_dl = TfmdDL(
        HFtoFAI(valid_ds), bs=128, shuffle=False,
        after_item=item_tfms, after_batch=valid_batch_tfms,
        num_workers=1, device='cpu',
    )

    dls = DataLoaders(train_dl, valid_dl)
    dls.c = 10
    dls.vocab = list(range(10))


    learn = vision_learner(
        dls, resnet18,
        pretrained=False,
        n_out=10,
        loss_func=CrossEntropyLossFlat(),
        metrics=[accuracy, error_rate],
        cbs=[EarlyStoppingCallback(patience=3)],
    ).to_fp16()

    learn.fit_one_cycle(30)

    
    val_res = learn.validate()
    val_loss, val_acc, val_err = val_res
    print(f'==== End Fold {fold+1} with mean acc={val_acc:.3f} ====')

    fold_metrics.append({
        'fold': fold,
        'val_loss': float(val_loss),
        'acc': float(val_acc),
        'err': float(val_err),
    })
    learners.append(learn)

print('==== Summary ====')
print('Mean accuracy:', np.mean([m['acc'] for m in fold_metrics]))
print('Std accuracy:', np.std([m['acc'] for m in fold_metrics]))


==== Fold 1 ====


/home/linux/tuke/zs2025/deep_learning/env/lib/python3.13/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,2.583992,1.907416,0.364000,0.636000,02:57
1,2.034530,1.619279,0.441300,0.558700,02:56
2,1.611417,1.387483,0.498700,0.501300,03:01
3,1.415727,1.226330,0.567100,0.432900,03:00
4,1.296909,1.266184,0.565500,0.434500,03:01
5,1.229784,1.376550,0.544200,0.455800,02:52
6,1.138074,1.016678,0.633200,0.366800,02:49
7,1.038641,1.050419,0.650900,0.349100,02:50
8,0.954033,0.914216,0.689400,0.310600,02:50
9,0.889096,0.894890,0.688400,0.311600,02:50


No improvement since epoch 20: early stopping


/home/linux/tuke/zs2025/deep_learning/env/lib/python3.13/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


...

No improvement since epoch 25: early stopping


==== End Fold 4 with mean acc=0.821 ====
==== Fold 5 ====


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,2.554208,1.941119,0.369400,0.630600,03:00
1,2.046623,1.643749,0.439100,0.560900,02:51
2,1.653730,1.496313,0.488900,0.511100,02:49
3,1.450456,1.419786,0.519600,0.480400,02:47
4,1.297464,1.188523,0.591300,0.408700,02:47
5,1.169388,1.200063,0.597800,0.402200,02:46
6,1.080171,1.011576,0.646900,0.353100,02:46
7,1.004274,0.969997,0.669300,0.330700,02:47
8,0.884769,0.930501,0.684600,0.315400,02:50
9,0.856195,0.805574,0.721200,0.278800,02:49


No improvement since epoch 23: early stopping


==== End Fold 5 with mean acc=0.828 ====
==== Summary ====
Mean accuracy: 0.8246399998664856
Std accuracy: 0.0037382230264735366


In [10]:
learn = learners[np.argmax([ _['acc'] for _ in fold_metrics]).item()]
learn

In [11]:
valid_ds = HFtoFAI(hf_valid)
valid_dl = TfmdDL(valid_ds, bs=1, shuffle=False,
                  after_item=item_tfms, after_batch=valid_batch_tfms,
                  num_workers=1, device=device,)
val_los, accuracy, error_rate = learn.validate(dl=valid_dl)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.821
